# WESM Price Prediction - EDA

### Setup and Loading

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')

df = pd.read_csv("final_dataset.csv")
df["datetime"] = pd.to_datetime(df["datetime"])

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15260 entries, 0 to 15259
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   datetime            15260 non-null  datetime64[ns]
 1   GWAP                15260 non-null  float64       
 2   energy_demand_mw    15260 non-null  float64       
 3   energy_supply_mw    15260 non-null  float64       
 4   energy_shortage_mw  15260 non-null  float64       
 5   reserve_demand_mw   15260 non-null  float64       
 6   reserve_supply_mw   15260 non-null  float64       
 7   outage_count        15260 non-null  float64       
 8   GWAP_Lag_1          15260 non-null  float64       
 9   GWAP_Lag_12         15260 non-null  float64       
 10  GWAP_Lag_288        15260 non-null  float64       
dtypes: datetime64[ns](1), float64(10)
memory usage: 1.3 MB
None


## I. Dataset Description

### The Data

The final dataset is generated from ETL Processing which combines data from three different catagories, **GWAP**, **RTD Regional Summaries**, and **Outage Schedules**. The columns are defined as follows:

- `datetime`: Target End Time of Dispatch Interval
- `GWAP`: Generator Weighted Average Price
- `energy_demand_mw`: MW Requirement for Energy
- `energy_supply_mw`: MW Generation for Energy
- `energy_shortage_mw`: 
- `reserve_demand_mw`: MW Requirement for Reserve Commodities (Non-energy commodity type)
- `reserve_supply_mw`: MW Generation for Reserve Commodities (Non-energy commodity type)
- `outage_count`: Count of outages per datetime
    - Value of 0 - No outages occured at timestamp
- `GWAP_Lag_1`: GWAP value 5 minutes ago
- `GWAP_Lag_12`: GWAP value 1 hour ago
- `GWAP_Lag_288`: GWAP value at the same time yesterday (24 hours ago)

In [5]:
print(df.describe())
print(df.head())

                            datetime          GWAP  energy_demand_mw  \
count                          15260  15260.000000      15260.000000   
mean   2025-11-29 12:08:12.581913344   3096.999168       9293.101514   
min              2025-11-03 00:05:00  -9999.000000       5034.010000   
25%              2025-11-16 06:13:45   2042.165950       8260.635000   
50%              2025-11-29 12:07:30   2635.679900       9291.240000   
75%              2025-12-12 18:01:15   4218.161300      10379.442500   
max              2025-12-26 00:00:00  31997.080000      11992.110000   
std                              NaN   2799.447522       1279.685872   

       energy_supply_mw  energy_shortage_mw  reserve_demand_mw  \
count      15260.000000        15260.000000       15260.000000   
mean        9590.939531         -297.838017        1536.719790   
min         5096.560000         -761.620000         940.000000   
25%         8600.255000         -435.645000        1430.000000   
50%         9608.3950

As seen above, the minimum GWAP recorded is -9999. Initially, we thought this was a sentinel value (error code) but after observing neighboring data points (e.g., -9998 and -9500) and reading the [WESM Price Determination Methodology](https://www.wesm.ph/downloads/download/TWFya2V0IFJlcG9ydHM=/NTYw), we were able to determine that these were valid data.

According to the WESM Price Determination Methodology, this phenomenon is driven by "Excess Generation", where the market clears at or near the Offer Price Floor (currently set at -P10,000/MWh).
> "5.4.2 In the event of over-generation, the excess price shall be determined as the offer price floor."

The Logic:
- Over-generation: Happens when Supply > Demand.
- "Pay-to-Stay" Strategy: the -9,999 (just P1.00 above the floor) implies a strategic bid. Baseload generators (like Coal) have high startup costs, so they bid highly negative prices (essentially paying the market to keep running) rather than shutting down.
- The Result: The -9999.0 GWAP confirms that the grid was saturated enough to force prices down to the "desperation bids" of the large plants.

## II. Data Cleaning

### Check for Duplicates

Each row in the dataset is unique as its data is recorded every five minutes from November to December of the year 2025. This check is done to validate that there are no duplicate data, rows whose values are identical to another. As the results display an empty dataframe, this means that there were no duplicates found.

In [7]:
duplicates = df[df.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [datetime, GWAP, energy_demand_mw, energy_supply_mw, energy_shortage_mw, reserve_demand_mw, reserve_supply_mw, outage_count, GWAP_Lag_1, GWAP_Lag_12, GWAP_Lag_288]
Index: []


### Check for Missing Values

Although missing data from columns such as `outage_count` and those that resulted from the creation of the **lag feature** have already been handled during ETL Processing, this check is to validate that there are no other missing values in the dataset. As seen below, there appears to be no other values missing.

In [8]:
df.isnull().sum()

datetime              0
GWAP                  0
energy_demand_mw      0
energy_supply_mw      0
energy_shortage_mw    0
reserve_demand_mw     0
reserve_supply_mw     0
outage_count          0
GWAP_Lag_1            0
GWAP_Lag_12           0
GWAP_Lag_288          0
dtype: int64

### Check for Inconsistent Data Type

This check is to see if the data types for each column are consistent. All coulmns are shown to have consistent data types across the entire dataset.

In [9]:
df.dtypes

datetime              datetime64[ns]
GWAP                         float64
energy_demand_mw             float64
energy_supply_mw             float64
energy_shortage_mw           float64
reserve_demand_mw            float64
reserve_supply_mw            float64
outage_count                 float64
GWAP_Lag_1                   float64
GWAP_Lag_12                  float64
GWAP_Lag_288                 float64
dtype: object

In [10]:
flag = False

for col in df.columns:
    types = df[col].apply(type).nunique()
    if types > 1:
        print(f"Column '{col}' has multiple data types.")
        flag = True

if not flag:
    print(f"Dataset has consistent data types in each column.")

Dataset has consistent data types in each column.
